In [8]:
import chromadb
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

In [7]:
## 벡터DB 종류에 따라 수정 필요

def vectordb_targets(db_path:str):
    client = chromadb.PersistentClient(path=db_path)
    for collection in client.list_collections():
        data = collection.get(include=['metadatas'])
    lv1 = list(set([d['First Division'] for d in data["metadatas"]]))
    print(lv1)
    lv2 = list(set([d['Second Division'] for d in data["metadatas"]]))
    print(lv2)
    rag_target = lv1 + lv2
    rag_target.insert(0, "vectorstore")
    rag_target.insert(0, "vectordb")
    docs = ", ".join(rag_target)
    return docs

db_path = "./db/chroma_db_02"
db_index = vectordb_targets(db_path=db_path)
db_index

['PORT', 'MANUAL', 'Rules']
['Integrated Smart Ship(ISS)', 'Cryostar', 'ABS', 'MARPOL', 'LR', 'NK', 'KR', 'Common', 'Win GD', 'Port Regulation', 'DNV', 'BV', 'SOLAS']


'vectordb, vectorstore, PORT, MANUAL, Rules, Integrated Smart Ship(ISS), Cryostar, ABS, MARPOL, LR, NK, KR, Common, Win GD, Port Regulation, DNV, BV, SOLAS'

In [3]:
# Data model
class RouteState(BaseModel):
    """Route a user query to the most relevant datasource."""
    datasource: Literal["similarity_search", "vectorstore", "web_search", "database"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore or a similarity or a database.",
    )

In [4]:
def routing_agent(state):
    global db_index

    system = f"""You are an expert at routing a user question to a vectorstore, web search or database.
    The vectorstore contains documents related to {db_index}, Use the vectorstore for questions on these topics. 
    The question contains words of similarity or sim search, Use similarity_search for the question.
    The question contains words related to database, Use the database for the question. 
    Otherwise, use web-search."""

    route_prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}"),])
    
    llm = ChatGroq(temperature=0, model_name= "llama-3.3-70b-versatile")    

    structured_llm_router = llm.with_structured_output(state)
    question_router = route_prompt | structured_llm_router
    return question_router

question_router = routing_agent(state=RouteState)
question_router

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert at routing a user question to a vectorstore, web search or database.\n    The vectorstore contains documents related to vectordb, vectorstore, PORT, MANUAL, Rules, Integrated Smart Ship(ISS), Cryostar, ABS, MARPOL, LR, NK, KR, Common, Win GD, Port Regulation, DNV, BV, SOLAS, Use the vectorstore for questions on these topics. \n    The question contains words of similarity or sim search, Use similarity_search for the question.\n    The question contains words related to database, Use the database for the question. \n    Otherwise, use web-search.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| RunnableBinding(bound=

In [5]:
query = 'with reference to "lr" rule, explain the measurement procedure of "noise"'
print(question_router.invoke({"question": query}))

datasource='vectorstore'


In [6]:
questions = ["who is Son Heung-min",
             "according to database, who is Son Heung-min",
             "according to vectordb, who is Son Heung-min",
             "check the recent performance data of Son Heung min",
             "sim search for this sentence. who is Son Heung-min",
             "find similar sentences for this sentence. who is Son Heung-min"]

for question in questions:
    result = question_router.invoke({'question': question})
    print(result)

datasource='web_search'
datasource='database'
datasource='vectorstore'
datasource='web_search'
datasource='similarity_search'
datasource='similarity_search'
